#Bronze Layer

In [0]:
spark.readStream
.format("cloudfiles")
.option("cloudFiles.format","json")

# Silver Layer

In [0]:
%sql
-------------------- Data Ingestion from Bronze Table-------------------
CREATE OR REFRESH STREAMING LIVE TABLE inhome_silver
------------Below are all data constraints or Data quality rules-----------------
(
    CONSTRAINT valid_date EXPECT (metrics_date > "2021-01-01") ON VIOLATION FAIL UPDATE, --Sample Filtering
    CONSTRAINT valid_date EXPECT (metrics_date > "2021-01-01") ON VIOLATION DROP ROW, --Sample Filtering
    CONSTRAINT valid_id EXPECT (line_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
    CONSTRAINT valid_operation EXPECT (operation IS NOT NULL) ON VIOLATION DROP ROW, -- For CDC
    CONSTRAINT valid_name EXPECT (deviceid IS NOT NULL or operation = "DELETE"), -- For CDC
    CONSTRAINT valid_address EXPECT (
    (address IS NOT NULL and 
    city IS NOT NULL and 
    state IS NOT NULL and 
    zip_code IS NOT NULL) or
    operation = "DELETE"), -- For CDC
    CONSTRAINT valid_email EXPECT (
    rlike(email, '^([a-zA-Z0-9_\\-\\.]+)@([a-zA-Z0-9_\\-\\.]+)\\.([a-zA-Z]{2,5})$') or 
    operation = "DELETE") ON VIOLATION DROP ROW
    )
------------------------------------------------------------------------------------
COMMENT "Append only data with 3 data quality checks"
TBLPROPERTIES ("quality" = "silver")
AS SELECT timestamp(order_timestamp) AS order_timestamp, --* EXCEPT (order_timestamp, source_file, _rescued_data)
FROM STREAM(LIVE.inhome_bronze)

# Gold Layer

In [0]:
%sql
CREATE OR REFRESH LIVE TABLE inhome_gold
AS 
SELECT *, "x" as aggregation_logic
FROM STREAM(LIVE.inhome_silver)

# Quaratine Records

In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE TABLE quarantine
(
  CONSTRAINT quaratine EXPECT (NOT(id > 0)) ON VIOLATION DROP ROW
)
AS SELECT *
FROM STREAM(<table_name>)